In [ ]:
from dolfinx import *
from dolfinx import default_scalar_type
from dolfinx.fem import *
from mpi4py import *
from dolfinx.fem.petsc import LinearProblem
from dolfinx.plot import vtk_mesh
from petsc4py import PETSc
import numpy as np
import matplotlib
import matplotlib.pyplot as pl
import math
import pyvista
import ufl
import basix

In [ ]:
# define exact solution
def u_exact_np(x):
    u = -1
    theta = np.arctan2(x[1],x[0])

    d_psi_d_r = (u/((1/4)*(np.pi**2)-1))*((-1/4)*(np.pi**2)*(np.sin(theta))+((1/2)*(np.pi)*(theta)*(np.sin(theta)))+((theta)*(np.cos(theta))))
    d_psi_d_theta_overR = ((u)/((1/4)*(np.pi**2)-1)*(((-1/4)*(np.pi**2)*np.cos(theta))+((1/2)*(np.pi)*(np.sin(theta)))+((1/2)*(np.pi)*(theta)*(np.cos(theta)))+(np.cos(theta))-((theta)*(np.sin(theta)))))

    return np.array([(((np.cos(theta) * (d_psi_d_theta_overR))) + (np.sin(theta) * d_psi_d_r)), (((np.sin(theta) * (d_psi_d_theta_overR))) - (np.cos(theta) * d_psi_d_r))])

In [ ]:
# define a function which returns values for Dirichlet boundary condition (60)
def u_0(x):
    u = -1
    us = u * np.ones_like(x[0])
    zeroes = np.zeros_like(x[0])
    
    return np.array([us, zeroes])

In [ ]:
#specify function inputs to test
ne = 4
p = 1

# define the domain (unit square) of length and width ne, running in parallel
domain = mesh.create_unit_square(MPI.COMM_WORLD, ne, ne)

# define velocity and pressure element
v_e = basix.ufl.element("Lagrange", domain.basix_cell(), p+1, shape=(domain.geometry.dim,))
p_e = basix.ufl.element("Lagrange", domain.basix_cell(), p)

# define mixed element
vPe = basix.ufl.mixed_element([v_e, p_e])

# define function space
V = functionspace(domain, vPe)

# define Velocity function space
V_v, _ = V.sub(0).collapse()

# define test functions
v_t, p_t = ufl.TestFunctions(V)

# define boundary value for Dirichlet boundary condition (59)
def boundary_D_1(x):
    return np.isclose(x[0], 0)

dofs_D_1 = locate_dofs_geometrical(V_v, boundary_D_1)
u_bc1 = Function(V_v)
u_bc1.x.array[:] = 0
bc1 = dirichletbc(u_bc1, dofs_D_1)

# define boundary value for Dirichlet boundary condition (60)
def boundary_D_2(x):
    return np.isclose(x[1], 0)

dofs_D_2 = locate_dofs_geometrical(V_v, boundary_D_2)
u_bc2 = Function(V_v)
u_bc2.interpolate(u_0)
bc2 = dirichletbc(u_bc2, dofs_D_2)

# define boundary value for Dirichlet boundary condition (61)
def boundary_D_3(x):
    return np.logical_or(np.isclose(x[0], 1), np.isclose(x[1], 1))

dofs_D_3 = locate_dofs_geometrical(V_v, boundary_D_3)
u_bc3 = Function(V_v)
u_bc3.interpolate(u_exact_np)
bc3 = dirichletbc(u_bc3, dofs_D_3)

In [ ]:
pyvista.start_xvfb()
pyvista_cells, cell_types, geometry = vtk_mesh(u_bc3.function_space)
grid = pyvista.UnstructuredGrid(pyvista_cells, cell_types, geometry)

values = np.zeros((geometry.shape[0], 3))
values[:, :len(u_bc3)] = u_bc3.x.array.real.reshape((geometry.shape[0], len(u_bc3)))
grid["u"] = values
geom = pyvista.Arrow()
glyphs = grid.glyph(orient="u", geom=geom, factor=0.1)

plotter = pyvista.Plotter(window_size=[800,800])
#plotter.add_mesh(grid, show_edges=True, show_scalar_bar=False, cmap='coolwarm')
plotter.add_mesh(glyphs, cmap='coolwarm', show_scalar_bar=True)
plotter.add_text("Intermediate Vector Case: g Solution", position="upper_edge", font_size=14, color="black")
plotter.view_xy()
plotter.show()